In [1]:
!wget --no-check-certificate \
    https://storage.googleapis.com/laurencemoroney-blog.appspot.com/horse-or-human.zip \
    -O /tmp/horse-or-human.zip

--2021-02-24 09:15:34--  https://storage.googleapis.com/laurencemoroney-blog.appspot.com/horse-or-human.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 2404:6800:4007:811::2010, 2404:6800:4007:808::2010, 2404:6800:4007:804::2010, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|2404:6800:4007:811::2010|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 149574867 (143M) [application/zip]
Saving to: ‘/tmp/horse-or-human.zip’

/tmp/horse-or-human 100%[===================>] 142.65M   471KB/s    in 7m 24s  

2021-02-24 09:22:59 (329 KB/s) - ‘/tmp/horse-or-human.zip’ saved [149574867/149574867]



In [2]:
import os
import zipfile

local_zip = '/tmp/horse-or-human.zip'
zip_ref = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall('/tmp/horse-or-human')
zip_ref.close()

In [3]:
train_horse_dir = os.path.join('/tmp/horse-or-human/horses')
train_human_dir = os.path.join('/tmp/horse-or-human/humans')

In [4]:
train_horse_names = os.listdir(train_horse_dir)
print(train_horse_names[:10])

train_human_names = os.listdir(train_human_dir)
print(train_human_names[:10])

['horse45-9.png', 'horse26-5.png', 'horse23-3.png', 'horse13-6.png', 'horse29-9.png', 'horse19-4.png', 'horse15-3.png', 'horse50-3.png', 'horse23-1.png', 'horse09-3.png']
['human05-11.png', 'human14-22.png', 'human08-23.png', 'human04-14.png', 'human05-15.png', 'human05-09.png', 'human14-23.png', 'human03-05.png', 'human07-25.png', 'human14-04.png']


In [5]:
print('total training horse images:', len(os.listdir(train_horse_dir)))
print('total training human images:', len(os.listdir(train_human_dir)))

total training horse images: 500
total training human images: 527


In [6]:
import tensorflow as tf

In [7]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(16, (3,3), activation='relu', input_shape=(300, 300, 3)),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Conv2D(32, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

In [8]:
from tensorflow.keras.optimizers import RMSprop

model.compile(loss='binary_crossentropy',
              optimizer=RMSprop(lr=0.001),
              metrics=['accuracy'])

In [9]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale=1/255)

train_generator = train_datagen.flow_from_directory(
        '/tmp/horse-or-human/',  
        target_size=(300, 300),  
        batch_size=128,
        class_mode='binary')


Found 1027 images belonging to 2 classes.


In [10]:
history = model.fit(
      train_generator,
      steps_per_epoch=8,  
      epochs=15,
      verbose=1)

Epoch 1/15
8/8 [==============================] - 78s 9s/step - loss: 1.0071 - accuracy: 0.5162
Epoch 2/15
8/8 [==============================] - 69s 8s/step - loss: 1.3708 - accuracy: 0.7089
Epoch 3/15
8/8 [==============================] - 68s 8s/step - loss: 0.6169 - accuracy: 0.6372
Epoch 4/15
8/8 [==============================] - 73s 9s/step - loss: 0.4024 - accuracy: 0.8558
Epoch 5/15
8/8 [==============================] - 71s 10s/step - loss: 0.3081 - accuracy: 0.9178
Epoch 6/15
8/8 [==============================] - 70s 8s/step - loss: 0.3351 - accuracy: 0.8933
Epoch 7/15
8/8 [==============================] - 62s 7s/step - loss: 0.1327 - accuracy: 0.9390
Epoch 8/15
8/8 [==============================] - 65s 8s/step - loss: 0.2682 - accuracy: 0.9045
Epoch 9/15
8/8 [==============================] - 64s 8s/step - loss: 0.1503 - accuracy: 0.9366
Epoch 10/15
8/8 [==============================] - 67s 8s/step - loss: 0.1758 - accuracy: 0.9339
Epoch 11/15
8/8 [=====================

In [17]:
import numpy as np
from tensorflow.keras.preprocessing import image
path = '/home/durkesh/Downloads/homeless-844206_1920.jpg'
img = image.load_img(path, target_size=(300, 300))
x = image.img_to_array(img)
x = np.expand_dims(x, axis=0)
images = np.vstack([x])
classes = model.predict(images, batch_size=10)
print(classes[0])
if classes[0]>0.5:
    print(path + " is a human")
else:
    print(path + " is a horse")

[1.]
/home/durkesh/Downloads/homeless-844206_1920.jpg is a human
